<a href="https://colab.research.google.com/github/zhus-dika/rec_system_predictions/blob/master/Hometask5_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install optuna
!pip install lightfm
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 7.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp39-cp39-linux_x86_64.whl size=875591 sha256=1515a0a9c02fbded39d66802c07ac64ed074cb796a09634dc7e8cc72cbe30555
  Stored in directory: /root/.cache/pip/wheels/d8/65/93/6ac8180274dc2e8f86ff326be62da1dfa55dc158fd45faba7d
Successfully built lightfm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Загрузка и предобработка данных

In [2]:
import math
import numpy as np
import pandas as pd
from scipy import sparse
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import optuna

Загрузим данные и проведем предобраотку данных как на семинаре.

In [ ]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

Traceback (most recent call last):
  File "/home/dinara/anaconda3/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/home/dinara/anaconda3/lib/python3.9/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/home/dinara/anaconda3/lib/python3.9/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/dinara/.kaggle. Or use the environment method.
unzip:  cannot find or open articles-sharing-reading-from-cit-deskdrop.zip, articles-sharing-reading-from-cit-deskdrop.zip.zip or articles-sharing-reading-from-cit-deskdrop.zip.ZIP.


In [10]:
articles_df = pd.read_csv("shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [11]:
interactions_df = pd.read_csv("users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [12]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [13]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
    "VIEW": 1.0,
    "LIKE": 2.0,
    "BOOKMARK": 2.5,
    "FOLLOW": 3.0,
    "COMMENT CREATED": 4.0,
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(
    lambda x: event_type_strength[x]
)

Оставляем только тех пользователей, которые провзамодействовали более чем с пятью статьями.

In [14]:
users_interactions_count_df = (
    interactions_df.groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId")
    .size()
)
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5
].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [15]:
interactions_from_selected_users_df = interactions_df.loc[
    np.in1d(interactions_df.personId, users_with_enough_interactions_df)
]

In [16]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглаживаем полученный результат, взяв от него логарифм.

In [17]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df.groupby(["personId", "contentId"])
    .eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = interactions_from_selected_users_df.groupby(
    ["personId", "contentId"]
)["timestamp"].last()

interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [19]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp < split_ts
].copy()
interactions_test_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp >= split_ts
].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df.sample(5)

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
33802,692689608292948411,-3780822597455574960,1.0,1469731828
16177,-7909814890969546668,-3716447017462787559,1.0,1473687732
951,-1130272294246983140,-3750879736572068916,1.0,1464115151
24024,2891568537595470094,-8377626164558006982,1.0,1474370409
23847,2785512916786763092,-3170783292462058942,1.0,1464201248


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [20]:
interactions = (
    interactions_train_df.groupby("personId")["contentId"]
    .agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = interactions_test_df.groupby("personId")["contentId"].agg(
    lambda x: list(x)
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    ""
    for x in range(
        len(interactions.loc[pd.isnull(interactions.true_test), "true_test"])
    )
]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

## Задание 1 (1.5 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [21]:
# Ваш код здесь
data_train = pd.pivot_table(
    interactions_full_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)
data_test = pd.pivot_table(
    interactions_full_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)

In [22]:
data_train.sample(5)

contentId,-1006791494035379303,-1021685224930603833,-1022885988494278200,-1024046541613287684,-1033806831489252007,-1038011342017850,-1039912738963181810,-1046621686880462790,-1051830303851697653,-1055630159212837930,...,9222265156747237864,943818026930898372,957332268361319692,962287586799267519,966067567430037498,967143806332397325,972258375127367383,980458131533897249,98528655405030624,991271693336573226
personId,,,,,,,,,,,,,,,,,,,,,
5030175185311671155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.584963,...,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8992729171160464416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5663484293775216130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-8550167523008133722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
-4095153241356673613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
for person in tqdm(list(interactions_full_df.personId.unique())):
    if person in list(interactions_train_df.personId.unique()):
        contents_df = interactions_train_df[interactions_train_df.personId == person]
        for paper in list(data_train.loc[person].index):
            if paper not in list(contents_df.contentId):
                data_train[paper][person] = 0.0
    else:
        for paper in list(data_train.loc[person].index):
            data_train[paper][person] = 0.0

100%|██████████| 1140/1140 [04:59<00:00,  3.81it/s]


In [24]:
for person in tqdm(list(interactions_full_df.personId.unique())):
    if person in list(interactions_test_df.personId.unique()):
        contents_df = interactions_test_df[interactions_test_df.personId == person]
        for paper in list(data_test.loc[person].index):
            if paper not in list(contents_df.contentId):
                data_test[paper][person] = 0.0
    else:
        for paper in list(data_test.loc[person].index):
            data_test[paper][person] = 0.0

100%|██████████| 1140/1140 [04:50<00:00,  3.92it/s]


In [25]:
data_train_sparse = sparse.csr_matrix(data_train.values)
data_train_sparse

<1140x2984 sparse matrix of type '<class 'numpy.float64'>'
	with 29329 stored elements in Compressed Sparse Row format>

In [26]:
data_test_sparse = sparse.csr_matrix(data_test.values)
data_test_sparse

<1140x2984 sparse matrix of type '<class 'numpy.float64'>'
	with 9777 stored elements in Compressed Sparse Row format>

## Задание 2 (0.5 балла)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [27]:
model = LightFM(loss='warp', random_state=123)
model.fit(data_train_sparse, epochs=10, num_threads=16)

In [28]:
train_precision = precision_at_k(model, data_train_sparse, k=10).mean()
test_precision = precision_at_k(model, data_test_sparse, k=10, train_interactions=data_train_sparse).mean()
print(train_precision)
print(test_precision)

0.21241008
0.0069246436


## Задание 3 (2 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [29]:
vectorizer = TfidfVectorizer()
papers = list(interactions_full_df.contentId.unique())
feat = vectorizer.fit_transform([np.array(articles_df.loc[articles_df['contentId'] == i].text)[0] if i in list(articles_df.contentId) else '' for i in papers])
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., '食べ物', '食事中', '건배'], dtype=object)

In [30]:
feat.shape

(2984, 71875)

In [46]:
model = LightFM(loss='warp', random_state=123)
model.fit(data_train_sparse, epochs=10, num_threads=16, item_features=feat)

In [47]:
train_precision = precision_at_k(model, data_train_sparse, k=10, item_features = feat).mean()
test_precision = precision_at_k(model, data_test_sparse, k=10, train_interactions=data_train_sparse, item_features = feat).mean()
print(train_precision)
print(test_precision)

0.20962231
0.006517312


## Задание 4 (1.5 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [33]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('snowball_data')
nltk.download('perluniprops')
nltk.download('universal_tagset')
nltk.download('nonbreaking_prefixes')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [34]:
articles_df['text_lower'] = articles_df.text.apply(lambda x: x.lower())
articles_df.sample(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,text_lower
94,1459369531,CONTENT SHARED,-5973686110278905367,4670267857749552625,204912088629071541,NaN,NaN,NaN,HTML,http://www.mckinsey.com/industries/high-tech/o...,The Internet of Things,More objects are becoming embedded with sensor...,en,more objects are becoming embedded with sensor...
1352,1465651032,CONTENT SHARED,2265761247530716287,6040926917589508037,4802031496467040625,NaN,NaN,NaN,HTML,http://arstechnica.com/information-technology/...,Snark attack: Cornell students teach software ...,A team of students participating in Cornell Un...,en,a team of students participating in cornell un...


In [35]:
from pymorphy2 import MorphAnalyzer
#from functools import lru_cache
from nltk.corpus import stopwords
import re

m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [36]:
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('english') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    return ' '.join(remove_stopwords(lemmas))

In [37]:
from multiprocessing import Pool


with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, articles_df['text_lower']), total=len(articles_df)))
    
articles_df['lemmas'] = lemmas
articles_df.sample(5)

100%|██████████| 3047/3047 [01:00<00:00, 50.08it/s]


,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,text_lower,lemmas
259,1460041957,CONTENT SHARED,-5179258585851879449,5660542693104786364,871944229415655122,NaN,NaN,NaN,HTML,http://techcrunch.com/2016/04/03/the-insurance...,The insurance tech equation,"Insurance policies can be complex, and some po...",en,"insurance policies can be complex, and some po...",insurance policies complex policyholders under...
1525,1466865784,CONTENT SHARED,-5762305367413370284,-6895155480127642372,6182198327666594200,NaN,NaN,NaN,HTML,http://www.cnseg.org.br/fenaprevi/previdencia-...,Previdência Complementar,PREVIDÊNCIA PRIVADA COMPLEMENTAR ABERTA No cur...,pt,previdência privada complementar aberta no cur...,previd ncia privada complementar aberta curso ...
936,1463101717,CONTENT SHARED,626121253933491500,-1443636648652872475,-6889636411309016105,NaN,NaN,NaN,HTML,http://venturebeat.com/2016/05/12/amazon-just-...,Amazon just released a Dash button for AWS IoT,"Believe it or not, there is now an Amazon Dash...",en,"believe it or not, there is now an amazon dash...",believe amazon dash button integrate managed s...
2492,1475504330,CONTENT SHARED,-7838000919054582150,-1130272294246983140,4393555317645464794,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0...,SP,BR,VIDEO,https://www.facebook.com/FestivalHerculeFloren...,Festival de Fotografia Hercule Florence - Time...,Você sabia que Campinas será tomada pela fotog...,pt,você sabia que campinas será tomada pela fotog...,sabia campinas tomada pela fotografia novembro...
3041,1485909279,CONTENT SHARED,-5608160619992456731,1811781596621121379,-7405177444167459514,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,HTML,http://www.businessinsider.com/pioneering-wome...,12 incredible women you've never heard of who ...,"Shutterstock Sure, most people have heard of M...",en,"shutterstock sure, most people have heard of m...",shutterstock sure people heard marie curie ros...


In [ ]:
🐂 Делаем предсказания с помощью LightFM после лемматизации

In [38]:
vectorizer = TfidfVectorizer()
papers = list(interactions_full_df.contentId.unique())
feat_lemmas = vectorizer.fit_transform([np.array(articles_df.loc[articles_df['contentId'] == i].lemmas)[0] if i in list(articles_df.contentId) else '' for i in papers])
vectorizer.get_feature_names_out()

array(['_____', '______', '_______', ..., 'соnvеr', 'соuрl', 'уеаr'],
      dtype=object)

In [79]:
feat_lemmas.shape

(2984, 63066)

In [42]:
model = LightFM(loss='warp', random_state=123)
model.fit(data_train_sparse, epochs=10, num_threads=16, item_features=feat_lemmas)

In [45]:
train_precision = precision_at_k(model, data_train_sparse, k=10, item_features = feat_lemmas).mean()
test_precision = precision_at_k(model, data_test_sparse, k=10, train_interactions=data_train_sparse, item_features = feat_lemmas).mean()
print(train_precision)
print(test_precision)

0.21915467
0.0065173116


Улучшилось ли качество предсказания?

In [ ]:
🦛 немного улучшилось

## Задание 5 (1.5 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [158]:
import itertools

import numpy as np

from lightfm.evaluation import auc_score


def sample_hyperparameters():
    """
    Yield possible hyperparameter choices.
    """

    while True:
        yield {
            "no_components": np.random.randint(16, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            "max_sampled": np.random.randint(5, 15),
            "num_epochs": np.random.randint(5, 50),
        }


def random_search(train, test, num_samples=10, num_threads=1):
    """
    Sample random hyperparameters, fit a LightFM model, and evaluate it
    on the test set.

    Parameters
    ----------

    train: np.float32 coo_matrix of shape [n_users, n_items]
        Training data.
    test: np.float32 coo_matrix of shape [n_users, n_items]
        Test data.
    num_samples: int, optional
        Number of hyperparameter choices to evaluate.


    Returns
    -------

    generator of (auc_score, hyperparameter dict, fitted model)

    """

    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams)
        model.fit(train, epochs=num_epochs, num_threads=num_threads)

        #score = auc_score(model, test, train_interactions=train, num_threads=num_threads).mean()
        
        score = precision_at_k(model, train, k=10).mean()
        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)

In [159]:
(score, hyperparams, model) = max(random_search(data_train_sparse, data_test_sparse, num_threads=2), key=lambda x: x[0])

print("Best score {} at {}".format(score, hyperparams))

Best score 0.6886690258979797 at {'no_components': 55, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.22433071605941982, 'item_alpha': 1.0850990438170277e-08, 'user_alpha': 9.14128074610707e-09, 'max_sampled': 9, 'num_epochs': 44}


In [160]:
model = LightFM(no_components= hyperparams['no_components'], learning_schedule=hyperparams['learning_schedule'], loss=hyperparams['loss'], 
                learning_rate=hyperparams['learning_rate'], item_alpha=hyperparams['item_alpha'], 
                user_alpha=hyperparams['user_alpha'], 
                max_sampled=hyperparams['max_sampled'],
               random_state=42)
model.fit(data_train_sparse, epochs=hyperparams['num_epochs'])
train_precision = precision_at_k(model, data_train_sparse, k=10).mean()
test_precision = precision_at_k(model, data_test_sparse, k=10, train_interactions=data_train_sparse).mean()
print(train_precision)
print(test_precision)

0.69433457
0.005600815


## Задание 6 (1 балл)

Реализуйте функции для вычисления следующих метрик:
* precision@k
* recall@k
* NDCG@k



In [149]:
n_users, n_items = data_train.shape
user_ids = list(data_train.index)
item_ids = list(data_train.columns)
user_ids_test = list(interactions.index)

In [197]:
def sample_recommendation(model, n_items, user_int):
    scores = model.predict(user_int, np.arange(n_items))
    top_items = np.argsort(-scores)
    return top_items

In [194]:
def convert_int2id(typeid, lst):
  if typeid == 'user':
    return [user_ids[i] for i in lst]
  if typeid == 'item':
    return [item_ids[i] for i in lst]

In [211]:
k = 10
my_precision_k_custom = 0
interactions['predictions'] = interactions['true_train']
for user_int in range(n_users):
  if user_ids[user_int] in user_ids_test:
    topk = sample_recommendation(model, n_items, user_int)[:k]
    resk = convert_int2id('item', topk)
    res = set(resk).intersection(set(interactions['true_test'][user_ids[user_int]]))
    if len(res) > 0:
      my_precision_k_custom += len(res) / min(k, len(interactions['true_test'][user_ids[user_int]]))
      print(f'person: {user_ids[user_int]} --> contents: {res}')
      interactions['predictions'][user_ids[user_int]] = list(res)
    else:
      interactions['predictions'][user_ids[user_int]] = list(res)
print('my_precision_k_custom =', my_precision_k_custom / len(user_ids_test))

person: -1561982036714087726 --> contents: {'5658521282502533116'}
person: -1724358244764242875 --> contents: {'8224860111193157980'}
person: -2376088187709738156 --> contents: {'-8208801367848627943'}
person: -2457136404620190048 --> contents: {'-6843047699859121724'}
person: -3381527457560577265 --> contents: {'4105873627900556540'}
person: -3834559600071441352 --> contents: {'-7564569805639737618'}
person: -5088875531913232577 --> contents: {'3367026768872537336'}
person: -7611460419696903236 --> contents: {'-2440247087447106971'}
person: -8699750646678621887 --> contents: {'3170775058142440102'}
person: 1097401149118884075 --> contents: {'1356221992133852808', '5338677278233757627'}
person: 1560134519462672730 --> contents: {'-8992803137960175254'}
person: 7181121382861702936 --> contents: {'-8208801367848627943'}
person: 8195788452563155020 --> contents: {'6031953227014493100'}
person: 8745559424774781326 --> contents: {'-6156751702010469220'}
person: 9105269044962898535 --> conte

In [201]:
def my_precision_k(column, k):
    return (
        interactions
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column][:k]))) /
            min(len(row['true_test']) + 0.001, k),
            axis=1)).mean()

In [202]:
my_precision_k('predictions', k)

0.0036001707044157087

In [214]:
k = 10
my_recall_k_custom = 0
interactions['predictions'] = interactions['true_train']
for user_int in range(n_users):
  if user_ids[user_int] in user_ids_test:
    top = sample_recommendation(model, n_items, user_int)
    topk = top[:k]
    res = convert_int2id('item', top)
    resk = res[:k]
    result = set(resk).intersection(set(interactions['true_test'][user_ids[user_int]]))
    resultfull = set(res).intersection(set(interactions['true_test'][user_ids[user_int]]))
    if len(result) > 0:
      my_recall_k_custom += len(result) / len(resultfull)
      print(f'person: {user_ids[user_int]} --> contents: {result}')
      interactions['predictions'][user_ids[user_int]] = list(result)
    else:
      interactions['predictions'][user_ids[user_int]] = list(result)
print('my_recall_k_custom =', my_recall_k_custom / len(user_ids_test))

person: -1561982036714087726 --> contents: {'5658521282502533116'}
person: -1724358244764242875 --> contents: {'8224860111193157980'}
person: -2376088187709738156 --> contents: {'-8208801367848627943'}
person: -2457136404620190048 --> contents: {'-6843047699859121724'}
person: -3381527457560577265 --> contents: {'4105873627900556540'}
person: -3834559600071441352 --> contents: {'-7564569805639737618'}
person: -5088875531913232577 --> contents: {'3367026768872537336'}
person: -7611460419696903236 --> contents: {'-2440247087447106971'}
person: -8699750646678621887 --> contents: {'3170775058142440102'}
person: 1097401149118884075 --> contents: {'1356221992133852808', '5338677278233757627'}
person: 1560134519462672730 --> contents: {'-8992803137960175254'}
person: 7181121382861702936 --> contents: {'-8208801367848627943'}
person: 8195788452563155020 --> contents: {'6031953227014493100'}
person: 8745559424774781326 --> contents: {'-6156751702010469220'}
person: 9105269044962898535 --> conte

In [209]:
def my_recall_k(column, k):
    return (
        interactions
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column][:k]))) /
            (len(set(row['true_test']).intersection(
                set(row[column]))) + 0.0001),
            axis=1)).mean()

In [210]:
my_recall_k('predictions', k)

0.0034735213403650177

In [220]:
import math
k = 10
my_ndcg_k_custom = 0
interactions['predictions'] = interactions['true_train']
for user_int in range(n_users):
  if user_ids[user_int] in user_ids_test:
    top = sample_recommendation(model, n_items, user_int)
    topk = top[:k]
    res = convert_int2id('item', top)
    resk = res[:k]
    for i in resk:
      if i in set(interactions['true_test'][user_ids[user_int]]):
        my_ndcg_k_custom += 1 / (math.log2(k) + 1)
    result = set(resk).intersection(set(interactions['true_test'][user_ids[user_int]]))
    if len(result) > 0:
      my_ndcg_k_custom += len(result) / math.log2(k + 1)
      print(f'person: {user_ids[user_int]} --> contents: {result}')
      interactions['predictions'][user_ids[user_int]] = list(result)
    else:
      interactions['predictions'][user_ids[user_int]] = list(result)
norm_val = sum([1 / math.log2(i + 1) for i in range(1, k + 1)])
print('my_ndcg_k_custom =', my_ndcg_k_custom / (norm_val * len(user_ids_test)))

person: -1561982036714087726 --> contents: {'5658521282502533116'}
person: -1724358244764242875 --> contents: {'8224860111193157980'}
person: -2376088187709738156 --> contents: {'-8208801367848627943'}
person: -2457136404620190048 --> contents: {'-6843047699859121724'}
person: -3381527457560577265 --> contents: {'4105873627900556540'}
person: -3834559600071441352 --> contents: {'-7564569805639737618'}
person: -5088875531913232577 --> contents: {'3367026768872537336'}
person: -7611460419696903236 --> contents: {'-2440247087447106971'}
person: -8699750646678621887 --> contents: {'3170775058142440102'}
person: 1097401149118884075 --> contents: {'1356221992133852808', '5338677278233757627'}
person: 1560134519462672730 --> contents: {'-8992803137960175254'}
person: 7181121382861702936 --> contents: {'-8208801367848627943'}
person: 8195788452563155020 --> contents: {'6031953227014493100'}
person: 8745559424774781326 --> contents: {'-6156751702010469220'}
person: 9105269044962898535 --> conte

## Задание 7 (1 балл)

Вычислите значения реализованных метрик для $k=10$ для лучшей полученной модели в предыдущих шагах.

Найдите уже реализованные варианты этих метрик в библиотеках lightfm и sklearn. Сравните полученные у вас значения метрик с результатами встроенных в библиотеки метрик.

In [ ]:
from sklearn.metrics import dcg_score

## Задание 8 (1 балл)

Реализуйте алгоритм ALS и примените его для решения задачи ноутбука.

**ALS**

Итак, поставлена задача построения модели со скрытыми переменными (latent factor model) для коллаборативной фильтрации:

$$ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \to \min_{P,Q}$$

Суммирование ведется по всем парам $(u, i),$ для которых известен рейтинг $r_{ui}$ (и только по ним), а $p_u, q_i$ – латентные представления пользователя~$u$ и товара $i$, соответственно, матрицы $P, Q$ получаются путем записывания по столбцам векторов $p_u, q_i$ соответственно.

Подход ALS (Alternating Least Squares) решает задачу, попеременно фиксируя матрицы $P$ и $Q$, — оказывается, что, зафиксировав одну из матриц, можно выписать аналитическое решение задачи для другой.

$$\nabla_{p_u} \bigg[ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)q_i = 0$$

Воспользовавшись тем, что $a^Tbc = cb^Ta$, получим
$$\sum_{i} r_{ui}q_i - \sum_i q_i q_i^T p_u = 0.$$

Тогда окончательно каждый столбец матрицы $P$ можно найти по формуле
$$p_u = \bigg( \sum_i q_i q_i^T\bigg)^{-1}\sum_ir_{ui}q_i \;\; \forall u,$$

аналогично для столбцов матрицы $Q$
$$q_i = \bigg( \sum_u p_u p_u^T\bigg)^{-1}\sum_ur_{ui}p_u \;\; \forall i.$$

Таким образом мы можем решать оптимизационную задачу, поочередно фиксируя одну из матриц $P$ или $Q$ и проводя оптимизацию по второй.

**Оригинальная статья c постановкой задачи для ALS на explicit feedback:**

* Bell, R.M. and Koren, Y., 2007, October. Scalable collaborative filtering with jointly derived neighborhood interpolation weights. In Seventh IEEE international conference on data mining (ICDM 2007) (pp. 43-52). IEEE.

**Оригинальная статья с ALS для implicit данных, которая стала более известной:**

* Hu, Y., Koren, Y. and Volinsky, C., 2008, December. Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE international conference on data mining (pp. 263-272). Ieee.


In [ ]:
# Ваш код здесь